# Sparse Matrices

Focus on the example "Solving large sparse linear systems."

For the presentation, **add notes to the code**, explain its meaning and functionality.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import sparse
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_context('notebook', font_scale=1.5)

## Creating a sparse matrix

There are many applications in which we deal with matrices that are mostly zeros. For example, a matrix representing social networks is very sparse - there are 7 billion people, but most people are only connected to a few hundred or thousand others directly. Storing such a social network as a sparse rather than dense matrix  will offer orders of magnitude reductions in memory requirements and corresponding speed-ups in computation.

### Coordinate format

The simplest sparse  matrix format is built from the coordinates and values of the non-zero entries.

#### From dense matrix

In [ ]:
A = np.random.poisson(0.2, (5,15)) * np.random.randint(0, 10, (5, 15))
A

In [ ]:
np.nonzero(A)

In [ ]:
rows, cols = np.nonzero(A)

In [ ]:
rows

In [ ]:
cols

In [ ]:
vals = A[rows, cols]

In [ ]:
vals

In [ ]:
X1 = sparse.coo_matrix(A)
X1

In [ ]:
print(X1)

#### From coordinates

Note that the (values, (rows, cols)) argument is a single tuple.

In [ ]:
X2 = sparse.coo_matrix((vals, (rows, cols)))
X2

In [ ]:
print(X2)

#### Convert back to dense matrix

In [ ]:
X2.todense()

## Solving large sparse linear systems

SciPy provides efficient routines for solving large sparse systems as for dense matrices. We will illustrate by calculating the page rank  for airports using data from the [Bureau of Transportation Statisitcs](http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236). The [PageRank](https://en.wikipedia.org/wiki/PageRank) algorithm is used to rank web pages for search results, but it can be used to rank any node in a directed graph (here we have airports instead of web pages). PageRank is fundamentally about finding the steady state in a Markov chain and can be solved as a linear system.

The update at each time step for the page rank $PR$ of a page $p_i$ is 

![i0](https://wikimedia.org/api/rest_v1/media/math/render/svg/8a8c0a807f62331cc1740dd6c0f28ac1809926c7)

The PageRank algorithm assumes that every node can be reached from every other node. To guard against case where a node has out-degree 0, we allow every node a small random chance of transitioning to any other node using a damping factor $R$. Then we solve  the linear system  to find the pagerank score $R$.

![i1](https://wikimedia.org/api/rest_v1/media/math/render/svg/6bb0f1469218a064274fd4691143e9ce64639dc2)

In matrix notation, this is

![i2](https://wikimedia.org/api/rest_v1/media/math/render/svg/96265e6c41318e793194287f36b5f929075bb876)

At steady state,

![i3](https://wikimedia.org/api/rest_v1/media/math/render/svg/65d2fed50688deaca4640b117c88a9e7a3c2ef0d)

and we can rearrange terms to solve for $R$

![i4](https://wikimedia.org/api/rest_v1/media/math/render/svg/985f19f0c6b69d3a8afb5acc38339ebe4915baa7)

In [ ]:
data = pd.read_csv('airports.csv', usecols=[0,1])

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
lookup = pd.read_csv('names.csv', index_col=0)

In [ ]:
lookup.shape

In [ ]:
lookup.head()

In [ ]:
import networkx as nx

#### Construct the sparse adjacency matrix

In [ ]:
g = nx.from_pandas_edgelist(data, source='ORIGIN_AIRPORT_ID', target='DEST_AIRPORT_ID')

In [ ]:
airports = np.array(g.nodes())
adj_matrix = nx.to_scipy_sparse_array(g)

#### Construct the transition matrix

In [ ]:
out_degrees = np.ravel(adj_matrix.sum(axis=1))
diag_matrix = sparse.diags(1 / out_degrees).tocsr()
M = (diag_matrix @ adj_matrix).T

In [ ]:
n = len(airports) 
d = 0.85 
I = sparse.eye(n, format='csc')
A = I - d * M
b = (1-d) / n * np.ones(n) # so the sum of all page ranks is 1

In [ ]:
A.todense()

In [ ]:
from scipy.sparse.linalg import spsolve

In [ ]:
r =  spsolve(A, b)
r.sum()

In [ ]:
idx = np.argsort(r)

In [ ]:
top10 = idx[-10:][::-1]
bot10 = idx[:10]

In [ ]:
df = lookup.loc[airports[top10]]
df['degree'] = out_degrees[top10]
df['pagerank']= r[top10]
df

In [ ]:
df = lookup.loc[airports[bot10]]
df['degree'] = out_degrees[bot10]
df['pagerank']= r[bot10]
df

#### Visualize the airport connections graph and label the top and bottom 5 airports by pagerank

In [ ]:
labels = {airports[i]: lookup.loc[airports[i]].str.split(':').str[0].values[0] 
          for i in np.r_[top10[:5], bot10[:5]]}
nx.draw(g, pos=nx.spring_layout(g), labels=labels, 
        node_color='blue', font_color='red', alpha=0.5,
        node_size=np.clip(5000*r, 1, 5000*r), width=0.1)